<a href="https://colab.research.google.com/github/kieranysm3/DATA301-Projext/blob/main/kyi12_Data301_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#library and code setup
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

!pip install gdelt

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:15 

In [2]:
#start spark local server
import sys, os
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

Starting by identifying candidates for analysis by downloading a list of S&P companies

Source = https://www.slickcharts.com/sp500

Copied the list into txt and uploaded it to collab


Loading dataset

In [3]:
# file = sc.textFile("List of S&P500.txt")
# dbg(file)

Start sorting data provided: Tone of company needs to be averaged by week to be positive or negative

Manipulating Daily tone data to get weekly averages


In [33]:
file1 = sc.textFile("Fastenal-Company-Daily-Tone-Data.csv")

In [35]:
#trying to make reusable code that can be used to convert tone data from gdelt into format i want for analysis
dbg(file1)
file2 = file1.map(lambda x: x.split(","))
dbg(file2)
file3 = file2.map(lambda x: (x[0], float(x[2])))
dbg(file3)
#now i have my daily tone data 
#i need to calculate weekly averages - date of the first and average of the other 7 days
#filter x[2] into groups of 7 then do average function?
file4 = file3.collect()

def get_weekly_tone(datalist,  chunk_size):
  #chunks the daily results into lists of size chunk (7)
  for i in range(0, len(datalist), chunk_size):
    yield datalist[i:i +chunk_size]

def weekly_average_tone(dataset):
  #for each chunk, get the average of each result
  result = 0
  for i in range(len(dataset)):
    result += dataset[i][1]
  return [result/7]

set1 = list(get_weekly_tone(file4, 7))
print(set1)
#now each set of data is grouped weekly
#convert back to rdd?
rdd = sc.parallelize(set1)
print("printing rdd")
dbg(rdd)

rdd2 = rdd.flatMap(lambda x: (weekly_average_tone(x)))
dbg(rdd2)
#collect the file data to be written out
outfile = rdd2.collect()
rdd2.coalesce(1).saveAsTextFile("/content/Fastanal-Weekly-Average-Tone.csv")
# df.write.format("csv").mode("overwrite).save(outputPath/apple.csv)

  

['29/4/2019,Average Tone,-0.1213', '30/4/2019,Average Tone,0', '1/5/2019,Average Tone,0.5608', '2/5/2019,Average Tone,0.8132', '3/5/2019,Average Tone,0.3282', '4/5/2019,Average Tone,1.0954', '5/5/2019,Average Tone,0', '6/5/2019,Average Tone,0.5668', '7/5/2019,Average Tone,1.598', '8/5/2019,Average Tone,0', '9/5/2019,Average Tone,0.7538', '10/5/2019,Average Tone,-1.1983', '11/5/2019,Average Tone,1.271', '12/5/2019,Average Tone,0', '13/5/2019,Average Tone,0.6389', '14/5/2019,Average Tone,0', '15/5/2019,Average Tone,2.1431', '16/5/2019,Average Tone,0.1628', '17/5/2019,Average Tone,0.8641', '18/5/2019,Average Tone,0', '19/5/2019,Average Tone,1.6796', '20/5/2019,Average Tone,-0.299', '21/5/2019,Average Tone,0.093', '22/5/2019,Average Tone,0', '23/5/2019,Average Tone,0.8606', '24/5/2019,Average Tone,-0.368', '25/5/2019,Average Tone,1.7554', '26/5/2019,Average Tone,0', '27/5/2019,Average Tone,2.753', '28/5/2019,Average Tone,0', '29/5/2019,Average Tone,0.3055', '30/5/2019,Average Tone,0', '31/

This section of code will be for the manipulation of datasets to see how accurate the tone was at predicting the stock prices each week

In [38]:
#load in 2 different files. The weekly Tone data data and the Weekly stock market results
#headers have already been removed from these datasets
weekly_Stock_Data = sc.textFile("Fastenal-Weekly-Stock-Data.csv")
weekly_Tone_Averages = sc.textFile("/content/Fastanal-Weekly-Average-Tone.csv/Fastenal-Company-Daily-Tone-Data.csv")
#first split the csv strings into individual indexes
dbg(weekly_Stock_Data)
formatted_Weekly_Stock_Data = weekly_Stock_Data.map(lambda x: x.split(","))
dbg(formatted_Weekly_Stock_Data)
#Extract the values of the 5th row in dataset which show if stock price went up or down
# dbg(weekly_Stock_Data)

price_Movement_rdd = formatted_Weekly_Stock_Data.map(lambda x: x[8])
weekly_Tone_Averages = weekly_Tone_Averages.map(lambda x : x)
print("price_movement_data is: ")
dbg(price_Movement_rdd)
print("weekly tone averages are: ")
dbg(weekly_Tone_Averages)
merged = price_Movement_rdd.zip(weekly_Tone_Averages)
# rdd1.join(rdd2).map(case (k, (ls, rs)) => (k, ls ++ rs))
print("merged rdd is: ")
dbg(merged)

['6/5/2019,33.465,34.09,32.215,32.82,30.988392,30632600,0.645,1', '13/5/2019,32.055,32.794998,31.575001,32.084999,30.294407,21195400,-0.029999,-1', '20/5/2019,31.6,32.779999,30.719999,30.860001,29.137777,20444400,0.739999,1', '27/5/2019,30.91,31.129999,29.98,30.59,28.882843,22362000,0.32,1', '3/6/2019,30.01,32.59,30.01,32.259998,30.459642,22813300,-2.249998,-1', '10/6/2019,32.43,33.029999,31.129999,31.65,29.883686,13955900,0.78,1', '17/6/2019,31.76,33.240002,31.299999,32.599998,30.780666,22916900,-0.839998,-1', '24/6/2019,32.610001,32.830002,31.84,32.59,30.771233,18854100,0.020001,1', '1/7/2019,32.919998,33.09,31.68,32.09,30.299131,11125400,0.829998,1', '8/7/2019,32.049999,32.130001,29.440001,30.950001,29.222757,43903300,1.099998,1', '15/7/2019,30.98,31.389999,30.040001,30.370001,28.675123,20717000,0.609999,1', '22/7/2019,30.52,32.080002,30.5,31.629999,29.864801,21397600,-1.109999,-1', '29/7/2019,31.6,31.83,28.799999,29.620001,28.162487,26379300,1.979999,1', '5/8/2019,28.99,29.870001,2

Py4JJavaError: ignored

In [ ]:
#now i have the result if the stock price increased or decreased as well as 
#the average tone of the stock for the week 
dbg(merged)
#The result i want is if the increase/decrease in stock price was the same as the positivity/negativity of the tone
stock_change_and_weekly_avg_tone = merged
#convert strings to integers/floats
stock_change_and_weekly_avg_tone = stock_change_and_weekly_avg_tone.map(lambda x: (float(x[0]), float(x[1])))
dbg(stock_change_and_weekly_avg_tone)
def sign_checker(float1, float2):
  #if both are positive or both are negative, return 1 else return 0
  if (float1 > 0 and float2 > 0):
    return 1
  if (float1 < 0 and float2 < 0):
    return 1
  else:
    return 0
result = stock_change_and_weekly_avg_tone.map(lambda x: sign_checker(x[0], x[1]))
dbg(result)
final = result.collect()
print(sum(final))
print(len(final))
#collecting the results to do cosine similarity later
result.coalesce(1).saveAsTextFile("/content/Mettler-Toledo-International-Inc-Results.csv")


3rd part of the project. After calculating the values of weather the changes in stock prices were predicted by the tone of their articles, we want to see if there is a difference between the more popular stocks and the less popular stocks. Thus we are running a cosine similarity algorithm between them to see if we can spot a trend


In [19]:
conf = SparkConf().setMaster("local[1]").set("spark.executor.memory", "1g")
conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g") 

import operator
import math

def dot_product2(v1, v2):
    return sum(map(operator.mul, v1, v2))


def cosine_similarity(list1, list2):
    prod = dot_product2(list1, list2)
    len1 = math.sqrt(dot_product2(list1, list1))
    len2 = math.sqrt(dot_product2(list2, list2))
    return prod / (len1 * len2)

In [41]:
#loading in all the results gathered so far
Apple_results = sc.textFile("Apple-Final-Results-From-Rdd.csv").collect()
Mettler_Toledo_results = sc.textFile("Mettler-Toledo-International-Inc-Results.csv").collect()
Perrigo_Results = sc.textFile("Perrigo-Final-Result.csv").collect()
Fastenel_Results = sc.textFile("Fastenel-Final-Results.csv").collect()
#convert all strings to integers

dbg(Mettler_Toledo_results)
# cosine_similarity(Perrigo_Results, Mettler_Toledo_results)

['1', '0', '0', '1', '0', '1', '0', '0', '0', '1', '1', '1', '1', '0', '1', '1', '0', '0', '0', '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '0', '0', '0', '1', '1', '1', '0', '0', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '0', '1', '0', '1', '0', '1', '0', '1', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1']


In [44]:
import time
# conf = SparkConf().setMaster("local[1]").set("spark.executor.memory", "1g")
conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g") 
time_start = time.time()
Apple_results = list(map(int, Apple_results))
Apple_results_Rdd = sc.parallelize(Apple_results)
Mettler_Toledo_results = list(map(int, Mettler_Toledo_results))
Mettler_Toledo_results_Rdd = sc.parallelize(Mettler_Toledo_results)
Perrigo_Results = list(map(int, Perrigo_Results))
Perrigo_Results_Rdd = sc.parallelize(Perrigo_Results)
Fastenel_Results = list(map(int, Fastenel_Results))
Fastenel_Results_Rdd = sc.parallelize(Fastenel_Results)



#collect data to broadcast out
Apple_results_Data = Apple_results_Rdd.collect()
print("this is Apple_results_Rdd")
dbg(Apple_results_Rdd)
print("this is Mettler_Toledo_results")
dbg(Mettler_Toledo_results)

Test_User_Data = Fastenel_Results

def rdd_cos_sim(list1):
  #make string values in dataset to list of integers
  #get the cosine Similarity value using the list and the broadcasted list
  value = cosine_similarity(list1, broadcast_Test_User_List.value )
  return (list1, value)


#start by broadcasting the test user Data
broadcast_Test_User_List = sc.broadcast(Test_User_Data)
print("this is broadcasted values")
print(broadcast_Test_User_List.value)
result1 = rdd_cos_sim(Mettler_Toledo_results)

# result1 = Mettler_Toledo_results_Rdd.flatMap(lambda x: [rdd_cos_sim(x)])
print("result1")
dbg(result1)
# result2 = result1.sortBy(lambda x: x[1], ascending = False)
# print("carry over variable is \n")

time_end = time.time()
print("elapsed time is %s" % str(time_end-time_start))

# carry_over_variable = result2.take(5)
# for i in carry_over_variable:
#   print (i[1])

this is Apple_results_Rdd
[0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
this is Mettler_Toledo_results
[1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]
this is broadcasted values
[1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0